# COVID Modeling 

In [821]:
# data handling
import pandas as pd
import numpy as np

# seaborn and matplotlib for visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [822]:
train = pd.read_csv('../data/training_data.csv', index_col="Unnamed: 0")
train['Date'] = pd.to_datetime(train['Date'])
val = pd.read_csv('../data/validation_data.csv', index_col="Unnamed: 0")
test = pd.read_csv('../data/testing_data.csv', index_col="Unnamed: 0")

In [823]:
train.columns

Index(['Date', 'State', 'Total Pop', 'Day_of_Wk', 'Confirmed',
       'Confirmed_diff', 'Confirmed_rate', 'Confirmed_rate_diff', 'Deaths',
       'Deaths_diff', 'Deaths_rate', 'Deaths_rate_diff', 'Recovered',
       'Recovered_rate', 'Recovered_diff', 'Recovered_rate_diff', 'Active',
       'Active_diff', 'Active_rate_diff', 'Active_rate', 'Case_Fatality_Ratio',
       'Administered', 'Series_Complete_Yes', 'Month', 'Year',
       'Monthly Temp (F)', 'Monthly Avg Temp (F)'],
      dtype='object')

In [824]:
train.head()

,Date,State,Total Pop,Day_of_Wk,Confirmed,Confirmed_diff,Confirmed_rate,Confirmed_rate_diff,Deaths,Deaths_diff,...,Active_diff,Active_rate_diff,Active_rate,Case_Fatality_Ratio,Administered,Series_Complete_Yes,Month,Year,Monthly Temp (F),Monthly Avg Temp (F)
0,2020-04-12,Alabama,4903185,Sunday,3667,NaN,0.000748,NaN,93,NaN,...,NaN,NaN,0.000708,2.610160,0.0,0.0,4,2020,61.55,63.096875
1,2020-04-13,Alabama,4903185,Monday,3870,203.0,0.000789,0.000041,99,6.0,...,165.0,0.000034,0.000741,2.651312,0.0,0.0,4,2020,61.55,63.096875
2,2020-04-14,Alabama,4903185,Tuesday,4041,171.0,0.000824,0.000035,114,15.0,...,204.0,0.000042,0.000783,2.883886,0.0,0.0,4,2020,61.55,63.096875
3,2020-04-15,Alabama,4903185,Wednesday,4307,266.0,0.000878,0.000054,118,4.0,...,118.0,0.000024,0.000807,2.895706,0.0,0.0,4,2020,61.55,63.096875
4,2020-04-16,Alabama,4903185,Thursday,4465,158.0,0.000911,0.000032,133,15.0,...,255.0,0.000052,0.000859,3.060990,0.0,0.0,4,2020,61.55,63.096875


## Modeling Methods and Metrics

Since our modeling will take and predict data at the _state_ level, we want our metrics to also be computed at the state level. In this case, we want to compute the root mean squared error, so we aggregate our real and predicted values by state, compute the RSME within the state, and then average the RSME accross all states. 

In [825]:
def RSME_df(df, col_names): 
    '''
    df has two columns, one with predictions and one with actual values,
    passed as `col_names` (order irrelevant)
    Returns the RSME of the predictions w.r.t the actual values 
    '''
    return np.sqrt(np.mean((df[col_names[0]] - df[col_names[1]])**2))

def compute_RSME_by_state(model, X, Y):
    '''
    Y must have 'State' in the index 
    '''
    Y_pred = pd.DataFrame(data=model.predict(X), index=Y.index, columns=Y.columns)
    combined_data = Y_pred.merge(Y, left_index=True, right_index=True, suffixes=('_pred','_actual'))    
    
    state_pred = combined_data.groupby('State').agg(RSME_df, col_names = combined_data.columns)
    state_pred.columns = ['RSME',"_"]
    state_pred = state_pred[['RSME']]
    return state_pred

def avg_state_RSME(model, X, Y):
    state_RSMEs = compute_RSME_by_state(model, X, Y)
    return state_RSMEs.mean()

In [826]:
metrics = ['Avg RSME']
datasets = ["Training", "Validation"]


def compute_stats(model, X, Y):
    avg_RSME = avg_state_RSME(model, X, Y)
    # more computed values go here
    return (avg_RSME)

def compute_model_stats(model, X_train, Y_train, X_val, Y_val):
    model_stats = {}
    train_metrics = compute_stats(model, X_train, Y_train)
    model_stats['training'] = dict(zip(metrics, train_metrics))
    val_metrics = compute_stats(model, X_val, Y_val)
    model_stats['validation'] =  dict(zip(metrics, val_metrics))
    return model_stats
    
def print_model_stats(model_stats):
    print("Model Statistics:")
    print('                | ',' | '.join(list(model_stats.keys()), ))
    print('-----------------------------------------')
    for var in model_stats['training'].keys():
        print("{var:<15} |   {train:.3f}   |   {val:.3f}".format(var = var,
                                      train = model_stats['training'][var], 
                                      val   = model_stats['validation'][var]))


In [827]:
iterables = [datasets, metrics]
col_idx = pd.MultiIndex.from_product(iterables, names=["", ""])
    
def make_fresh_record():
    record = pd.DataFrame(columns=col_idx)
    record.index.name = "Model"
    return record 

def record_model_stats(record, model_stats, model_name, override=False): 
    model_stats_df = pd.json_normalize(model_stats, sep='_')
    model_stats_df.columns = col_idx
    model_data = model_stats_df.iloc[0]
    model_data.name = model_name
    new_record = record.copy()
    # override or new entry
    if override or model_name not in record.index:
        new_record.loc[model_name,:] = model_data
    #exists and don't overide 
    else:
        print("Warning: A model with the name '{}' already exists in this record.".format(model_name))
        print("         Either change model_name or set 'override=True'.")
        return record
    return new_record

def make_and_record_model(record, processing_fun, train, val, name, override=False):
    model, (X_train, Y_train), (X_val, Y_val) = processing_fun(train, val)
    model_stats = compute_model_stats(model, X_train, Y_train, X_val, Y_val)
    record = record_model_stats(record, model_stats, name, override)
    return (record, model, model_stats, {"train_data": (X_train, Y_train), "val_data":(X_val, Y_val)})

### COVID Cases Modeling



In [828]:
def relabel_timeseries_data(X, Y, W, col_name="input"):
    timeseries_names = [col_name+'_day_'+str(i) for i in range(1-W,1)]

    target_day = Y.name
    Y.name = 'target_'+col_name
    Y = Y.reset_index()
    X = X.set_axis(timeseries_names, axis=1, inplace=False)
    X['Target_day'] = target_day
    Y['Target_day'] = target_day
    X = X.reset_index()
    X = X.set_index(['Target_day','State'])
    Y = Y.set_index(['Target_day','State'])
    return (X, Y)

def create_timeseries(df, col):
    return df.pivot_table(index = 'State', columns='Date',
                   values=col).sort_values(by = 'Date', axis='columns')


def convert_timeseries_to_data(df, W, col_name='input'):
    '''
    df is a dataframe, with columns sorted in increasing order by date
    splits rows into timeseries data with W columns of 'input' associated 
    with the W+1 column of 'output' and combined for all rows 
    '''
    d = df.shape[1]
    X = df.iloc[:, 0:W]
    Y = df.iloc[:,W]
    X, Y = relabel_timeseries_data(X, Y, W, col_name)

    for i in range(1,d-W):#1,3,..., d-W-1
        X_data = df.iloc[:, i:i+W] # i+W-1 = W+1,W+2,... d-1
        Y_data = df.iloc[:,i+W] # i+W = W+2,W+3,..., W+d-W = d
        X_data, Y_data = relabel_timeseries_data(X_data, Y_data, W, col_name)
        X = X.append(X_data)
        Y = Y.append(Y_data)

    return (X, Y)

### Model 1 

Feed in confirmed cases for the previous 14 days (since 2 weeks is a standard COVID incubation period) and predict confirmed cases for the next day. 

In [829]:
from sklearn.linear_model import LinearRegression

def model_1_pipeline(data, test_data=False): 
    window_size = 14
    conf_timeseries_data = create_timeseries(data,'Confirmed_diff')
    X_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_diff")
    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_1_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_1_pipeline(train)
    X_val, Y_val = model_1_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [830]:
model_record = make_fresh_record()

In [831]:
model_record, m1, m1_stats, m1_data = make_and_record_model(model_record, 
                                                  model_1_processing, train, val, 
                                                  "Confirmed_diff only, 14D")
model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only, 14D",593.976453,1432.701763


### Model 2 

In [832]:
def model_2_pipeline(data, test_data=False): 
    window_size = 14
    active_timeseries_data = create_timeseries(data,'Active_diff')
    X_active_data, _  = convert_timeseries_to_data(active_timeseries_data, 
                                                window_size, 
                                                col_name="Active_diff")
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    _, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    #X_data = X_active_data.merge(X_conf_data, left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_active_data, Y_data)

def model_2_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_2_pipeline(train)
    X_val, Y_val = model_2_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [833]:
model_record, m2, m2_stats, m2_data = make_and_record_model(model_record, 
                                                  model_2_processing, train, val, 
                                                  "Active_diff only, 14D", override=True)
model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only, 14D",593.976453,1432.701763
"Active_diff only, 14D",988.687519,2680.69719


### Model 3

In [834]:
def model_3_pipeline(data, test_data=False): 
    window_size = 14
    active_timeseries_data = create_timeseries(data,'Active_diff')
    X_active_data, _  = convert_timeseries_to_data(active_timeseries_data, 
                                                window_size, 
                                                col_name="Active_diff")
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_active_data.merge(X_conf_data, left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_3_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_3_pipeline(train)
    X_val, Y_val     = model_3_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [835]:
model_record, m3, m3_stats, m3_data = make_and_record_model(model_record, 
                                                  model_3_processing, train, val, 
                                                  "Active_diff & Conf_diff, 14D", override=True)
model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only, 14D",593.976453,1432.701763
"Active_diff only, 14D",988.687519,2680.69719
"Active_diff & Conf_diff, 14D",594.104388,1479.993138


### Model 4

In [836]:
def model_4_pipeline(data, test_data=False): 
    window_size = 14
    recovered_timeseries_data = create_timeseries(data.fillna(0),'Recovered')
    X_rec_data, _  = convert_timeseries_to_data(recovered_timeseries_data, 
                                                window_size, 
                                                col_name="Recovered")
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_rec_data.merge(X_conf_data, left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_4_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_4_pipeline(train)
    X_val, Y_val     = model_4_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [837]:
model_record, m4, m4_stats, m4_data = make_and_record_model(model_record, 
                                                  model_4_processing, train, val, 
                                                  "Recovered & Conf_diff, 14D", override=True)
model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only, 14D",593.976453,1432.701763
"Active_diff only, 14D",988.687519,2680.69719
"Active_diff & Conf_diff, 14D",594.104388,1479.993138
"Recovered & Conf_diff, 14D",592.739733,1475.218196


### Model 5 

In [838]:
train.head()

,Date,State,Total Pop,Day_of_Wk,Confirmed,Confirmed_diff,Confirmed_rate,Confirmed_rate_diff,Deaths,Deaths_diff,...,Active_diff,Active_rate_diff,Active_rate,Case_Fatality_Ratio,Administered,Series_Complete_Yes,Month,Year,Monthly Temp (F),Monthly Avg Temp (F)
0,2020-04-12,Alabama,4903185,Sunday,3667,NaN,0.000748,NaN,93,NaN,...,NaN,NaN,0.000708,2.610160,0.0,0.0,4,2020,61.55,63.096875
1,2020-04-13,Alabama,4903185,Monday,3870,203.0,0.000789,0.000041,99,6.0,...,165.0,0.000034,0.000741,2.651312,0.0,0.0,4,2020,61.55,63.096875
2,2020-04-14,Alabama,4903185,Tuesday,4041,171.0,0.000824,0.000035,114,15.0,...,204.0,0.000042,0.000783,2.883886,0.0,0.0,4,2020,61.55,63.096875
3,2020-04-15,Alabama,4903185,Wednesday,4307,266.0,0.000878,0.000054,118,4.0,...,118.0,0.000024,0.000807,2.895706,0.0,0.0,4,2020,61.55,63.096875
4,2020-04-16,Alabama,4903185,Thursday,4465,158.0,0.000911,0.000032,133,15.0,...,255.0,0.000052,0.000859,3.060990,0.0,0.0,4,2020,61.55,63.096875


In [839]:
def model_5_pipeline(data, test_data=False): 
    window_size = 14
    series_timeseries_data = create_timeseries(data.fillna(0),'Series_Complete_Yes')
    X_series_data, _  = convert_timeseries_to_data(series_timeseries_data, 
                                                window_size, 
                                                col_name="Series_Complete_Yes")
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_series_data.merge(X_conf_data, left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_5_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_5_pipeline(train)
    X_val, Y_val     = model_5_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [840]:
model_record, m5, m5_stats, m5_data = make_and_record_model(model_record, 
                                                  model_5_processing, train, val, 
                                                  "Series_complete & Conf_diff, 14D", override=True)
model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only, 14D",593.976453,1432.701763
"Active_diff only, 14D",988.687519,2680.69719
"Active_diff & Conf_diff, 14D",594.104388,1479.993138
"Recovered & Conf_diff, 14D",592.739733,1475.218196
"Series_complete & Conf_diff, 14D",593.976453,1432.701763


### Model 6

In [841]:
def model_6_pipeline(data, test_data=False): 
    window_size = 14
    admin_timeseries_data = create_timeseries(data.fillna(0),'Administered')
    X_vax_data, _  = convert_timeseries_to_data(admin_timeseries_data, 
                                                window_size, 
                                                col_name="Administered")
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_vax_data.merge(X_conf_data, left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_6_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_6_pipeline(train)
    X_val, Y_val     = model_6_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [842]:
model_record, m6, m6_stats, m6_data = make_and_record_model(model_record, 
                                                  model_6_processing, train, val, 
                                                  "Administered & Conf_diff, 14D", override=True)
model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only, 14D",593.976453,1432.701763
"Active_diff only, 14D",988.687519,2680.69719
"Active_diff & Conf_diff, 14D",594.104388,1479.993138
"Recovered & Conf_diff, 14D",592.739733,1475.218196
"Series_complete & Conf_diff, 14D",593.976453,1432.701763
"Administered & Conf_diff, 14D",593.976453,1432.701763


In [843]:
train.head()
train[['Administered', 'Series_Complete_Yes']].describe()

,Administered,Series_Complete_Yes
count,11417.0,11417.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


Both of these variables are entirely empty for the time covered by "training" data, so neither changes the model... from just having "Confirmed_diff". 

### Model 7

In [844]:
def model_7_pipeline(data, test_data=False): 
    window_size = 14
    recovered_timeseries_data = create_timeseries(data.fillna(0),'Recovered')
    X_rec_data, _  = convert_timeseries_to_data(recovered_timeseries_data, 
                                                window_size, 
                                                col_name="Recovered")
    
    active_timeseries_data = create_timeseries(data,'Active_diff')
    X_active_data, _  = convert_timeseries_to_data(active_timeseries_data, 
                                                window_size, 
                                                col_name="Active_diff")
    
    timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_conf_data.merge(X_rec_data, left_index=True, right_index=True)
    X_data = X_data.merge(X_active_data, left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_7_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_7_pipeline(train)
    X_val, Y_val     = model_7_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [845]:
model_record, m7, m7_stats, m7_data = make_and_record_model(model_record, 
                                                  model_7_processing, train, val, 
                                                  "Active, Recovered, & Conf_diff, 14D", override=True)
model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only, 14D",593.976453,1432.701763
"Active_diff only, 14D",988.687519,2680.69719
"Active_diff & Conf_diff, 14D",594.104388,1479.993138
"Recovered & Conf_diff, 14D",592.739733,1475.218196
"Series_complete & Conf_diff, 14D",593.976453,1432.701763
"Administered & Conf_diff, 14D",593.976453,1432.701763
"Active, Recovered, & Conf_diff, 14D",590.467725,1502.307988


Doesn't really help to include more, this is likely because of colinearity and not much new data being added. 

## Weather Models

First we should check for null values in our weather data.

In [846]:
null_temps_by_state = train[['State','Monthly Avg Temp (F)','Monthly Temp (F)']].set_index('State').isna().groupby(level=0).sum()
null_temps_by_state[null_temps_by_state.any(1)]

,Monthly Avg Temp (F),Monthly Temp (F)
State,,
District of Columbia,233,233


We see that D.C. has lots of null values, but all other states have data for all rows. We saw in our weather data analysis that D.C. was not included as a Division for weather collection. For our weather models we will exclude D.C. from analysis, and restrict to proper states. 

In [847]:
weather_train = train[train['State']!='District of Columbia']
weather_val   = val[val['State']!='District of Columbia']
weather_test  = test[test['State']!='District of Columbia']
null_temps_by_state = weather_train[['State','Monthly Avg Temp (F)','Monthly Temp (F)']].set_index('State').isna().groupby(level=0).sum()
display(null_temps_by_state[null_temps_by_state.any(1)])
print("Missing Weather Training Values:   ",weather_train[['State','Monthly Avg Temp (F)','Monthly Temp (F)']].isna().sum().sum())
print("Missing Weather Validation Values: ",weather_val[['State','Monthly Avg Temp (F)','Monthly Temp (F)']].isna().sum().sum())


,Monthly Avg Temp (F),Monthly Temp (F)
State,,


Missing Weather Training Values:    0
Missing Weather Validation Values:  0


After dropping D.C. we no longer have missing weather data. 

In [848]:
weather_model_record = make_fresh_record()
weather_model_record, m1_weather, m1_weather_stats, m1_weather_data = make_and_record_model(weather_model_record, 
                                                  model_1_processing, weather_train, weather_val, 
                                                  "Confirmed_diff only (weather), 14D")
weather_model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only (weather), 14D",605.225522,1460.577326


### Model 8 

Let's naively plug in the average monthly temperatures for each state based on the recent monthly averages (2000-2019). 

In [849]:
def model_8_pipeline(data, test_data=False): 
    window_size = 14
    
    X_temp_data = data[['State','Date','Monthly Avg Temp (F)']].set_index(['Date','State'])
    X_temp_data.index = X_temp_data.index.set_names('Target_day', level=0)
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_conf_data.merge(X_temp_data, how="left", left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_8_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_8_pipeline(train)
    X_val, Y_val     = model_8_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [850]:
weather_model_record, m8, m8_stats, m8_data = make_and_record_model(weather_model_record, 
                                                  model_8_processing, weather_train, weather_val, 
                                                  "Avg Mon. Temp & Conf_diff, 14D", override=True)
weather_model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only (weather), 14D",605.225522,1460.577326
"Avg Mon. Temp & Conf_diff, 14D",606.749975,1490.395993


### Model 9 
As we saw from our EDA (`analysis/data_merge_and_eda.ipynb`), absolute temperatures do not seem to follow a linear relationship with confirmed cases, however we did see increases for particularly cold or hot temperatures. To model this is our data, we can compute how far a temperature is from "room temperature", which we will take to be 70 degrees Fahrenheit. 

In [851]:
weather_train[['State','Monthly Avg Temp (F)','Monthly Temp (F)']].head()

,State,Monthly Avg Temp (F),Monthly Temp (F)
0,Alabama,63.096875,61.55
1,Alabama,63.096875,61.55
2,Alabama,63.096875,61.55
3,Alabama,63.096875,61.55
4,Alabama,63.096875,61.55


In [852]:
weather_train['month_diff_rm_temp'] = np.abs(weather_train['Monthly Temp (F)'] - 70)
weather_train['month_avg_diff_rm_temp'] = weather_train['Monthly Avg Temp (F)'] - 70
weather_train[['State','Monthly Avg Temp (F)','Monthly Temp (F)','month_diff_rm_temp','month_avg_diff_rm_temp']].head()


/var/folders/8n/v409d_pd7m18zx23825fgzsc0000gn/T/ipykernel_35902/691562054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_train['month_diff_rm_temp'] = np.abs(weather_train['Monthly Temp (F)'] - 70)
/var/folders/8n/v409d_pd7m18zx23825fgzsc0000gn/T/ipykernel_35902/691562054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_train['month_avg_diff_rm_temp'] = weather_train['Monthly Avg Temp (F)'] - 70


,State,Monthly Avg Temp (F),Monthly Temp (F),month_diff_rm_temp,month_avg_diff_rm_temp
0,Alabama,63.096875,61.55,8.45,-6.903125
1,Alabama,63.096875,61.55,8.45,-6.903125
2,Alabama,63.096875,61.55,8.45,-6.903125
3,Alabama,63.096875,61.55,8.45,-6.903125
4,Alabama,63.096875,61.55,8.45,-6.903125


In [853]:
def model_9_pipeline(data, test_data=False): 
    window_size = 14
    
    data.loc[:,'month_diff_rm_temp']     = np.abs(data['Monthly Temp (F)'] - 70)
    data.loc[:,'month_avg_diff_rm_temp'] = np.abs(data['Monthly Avg Temp (F)'] - 70)

    X_temp_data = data[['State','Date','month_avg_diff_rm_temp']].set_index(['Date','State'])
    X_temp_data.index = X_temp_data.index.set_names('Target_day', level=0)
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_conf_data.merge(X_temp_data, how="left", left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_9_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_9_pipeline(train)
    X_val, Y_val     = model_9_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [863]:
weather_model_record, m9, m9_stats, m9_data = make_and_record_model(weather_model_record, 
                                                  model_9_processing, weather_train, weather_val, 
                                                  "Abs Mon Avg Temp diff & Conf_diff, 14D", override=True)
weather_model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only (weather), 14D",605.225522,1460.577326
"Avg Mon. Temp & Conf_diff, 14D",606.749975,1490.395993
"Abs Mon Avg Temp diff & Conf_diff, 14D",605.533564,1453.493663


This doesn't really seem to improve the model. 

### Model 10 

We also saw that different states had different sensitivity to "cold" (e.g. Calfornia saw the cold effect between 40 and 50 degrees, but Vermont saw a less pronounced effect only below 40 degrees). We can incporate this into the data by computing the annual average temperature for each state and computing the difference for each month from the _state's_ annual average. 

In [864]:
hist_weather_data = pd.read_csv('../data/historical_monthly_temp_avgs_by_state.csv', index_col=0)
hist_weather_data.head()
state_avg_temps = hist_weather_data.groupby('State').mean().rename(columns={'Monthly Avg Temp (F)':"State Avg Temp"})[['State Avg Temp']]
state_avg_temps.head()

,State Avg Temp
State,
Alabama,63.428698
Arizona,63.623631
Arkansas,61.144722
California,57.130774
Colorado,46.451083


In [865]:
with_state_avg = weather_train.merge(state_avg_temps, how="left", left_on='State',right_index=True)
with_state_avg[['State','Monthly Avg Temp (F)','Monthly Temp (F)','State Avg Temp']]

,State,Monthly Avg Temp (F),Monthly Temp (F),State Avg Temp
0,Alabama,63.096875,61.55,63.428698
1,Alabama,63.096875,61.55,63.428698
2,Alabama,63.096875,61.55,63.428698
3,Alabama,63.096875,61.55,63.428698
4,Alabama,63.096875,61.55,63.428698
...,...,...,...,...
17220,Wyoming,30.490000,32.72,42.090167
17221,Wyoming,30.490000,32.72,42.090167
17222,Wyoming,30.490000,32.72,42.090167
17223,Wyoming,30.490000,32.72,42.090167


In [866]:
with_state_avg['Diff from Avg'] = np.abs(with_state_avg['Monthly Avg Temp (F)'] - with_state_avg['State Avg Temp'])
with_state_avg.sample(5)

,Date,State,Total Pop,Day_of_Wk,Confirmed,Confirmed_diff,Confirmed_rate,Confirmed_rate_diff,Deaths,Deaths_diff,...,Administered,Series_Complete_Yes,Month,Year,Monthly Temp (F),Monthly Avg Temp (F),month_diff_rm_temp,month_avg_diff_rm_temp,State Avg Temp,Diff from Avg
1928,2020-09-17,Connecticut,3565287,Thursday,55386,220.0,0.015535,0.000062,4488,1.0,...,0.0,0.0,9,2020,63.300000,64.048333,6.700000,5.951667,50.000278,14.048056
7240,2020-09-19,Michigan,9986857,Saturday,128087,587.0,0.012826,0.000059,6969,15.0,...,0.0,0.0,9,2020,58.490000,61.092500,11.510000,8.907500,45.892917,15.199583
11897,2020-11-13,Ohio,11689100,Friday,282528,8071.0,0.024170,0.000690,6080,71.0,...,0.0,0.0,11,2020,46.430000,42.460000,23.570000,27.540000,51.697667,9.237667
6762,2020-05-18,Massachusetts,6892503,Monday,87052,1042.0,0.012630,0.000151,5862,65.0,...,0.0,0.0,5,2020,55.833333,56.896667,14.166667,13.103333,48.642778,8.253889
1884,2020-08-04,Connecticut,3565287,Tuesday,50110,48.0,0.014055,0.000013,4437,0.0,...,0.0,0.0,8,2020,72.466667,70.821667,2.466667,0.821667,50.000278,20.821389


In [867]:
def model_10_pipeline(data, test_data=False): 
    window_size = 14
    
    with_state_avg = data.merge(state_avg_temps, how="left", left_on='State',right_index=True)

    with_state_avg['Diff from Avg'] = np.abs(with_state_avg['Monthly Avg Temp (F)'] - with_state_avg['State Avg Temp'])


    X_temp_data = with_state_avg[['State','Date','Diff from Avg']].set_index(['Date','State'])
    X_temp_data.index = X_temp_data.index.set_names('Target_day', level=0)
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_conf_data.merge(X_temp_data, how="left", left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_10_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_10_pipeline(train)
    X_val, Y_val     = model_10_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [868]:
weather_model_record, m10, m10_stats, m10_data = make_and_record_model(weather_model_record, 
                                                  model_10_processing, weather_train, weather_val, 
                                                  "Temp diff from State Avg & Conf_diff, 14D", override=True)
weather_model_record

,Training,Validation
,Avg RSME,Avg RSME
Model,,
"Confirmed_diff only (weather), 14D",605.225522,1460.577326
"Avg Mon. Temp & Conf_diff, 14D",606.749975,1490.395993
"Abs Mon Avg Temp diff & Conf_diff, 14D",605.533564,1453.493663
"Temp diff from State Avg & Conf_diff, 14D",605.533564,1453.493663


In [871]:
m10_data['train_data'][0].columns

Index(['Confirmed_Diff_day_-13', 'Confirmed_Diff_day_-12',
       'Confirmed_Diff_day_-11', 'Confirmed_Diff_day_-10',
       'Confirmed_Diff_day_-9', 'Confirmed_Diff_day_-8',
       'Confirmed_Diff_day_-7', 'Confirmed_Diff_day_-6',
       'Confirmed_Diff_day_-5', 'Confirmed_Diff_day_-4',
       'Confirmed_Diff_day_-3', 'Confirmed_Diff_day_-2',
       'Confirmed_Diff_day_-1', 'Confirmed_Diff_day_0', 'Diff from Avg'],
      dtype='object')

In [870]:
m10.coef_

array([[ 0.06170532, -0.08717308, -0.00655125, -0.05859181, -0.05866639,
        -0.04328052,  0.0829892 ,  0.29299939,  0.13738726,  0.12648172,
         0.15747023,  0.14616718,  0.13515263,  0.11050953, -4.81900588]])

### Model 11 

We saw that colder weather seemed to have a larger impact that warmer weather. To try and capture this in our model, let's compute a feature that measures how distance below average temperature, with values above average zeroed out. 

In [882]:
def zero_positive_values(val):
    if val < 0:
        return val
    else:
        return 0
    
def model_11_pipeline(data, test_data=False): 
    window_size = 14
    
    with_state_avg = data.merge(state_avg_temps, how="left", left_on='State',right_index=True)

    with_state_avg['Diff from Avg'] = with_state_avg['Monthly Avg Temp (F)'] - with_state_avg['State Avg Temp']
    with_state_avg['Diff below Avg'] = with_state_avg['Diff from Avg'].apply(zero_positive_values)

    X_temp_data = with_state_avg[['State','Date','Diff from Avg']].set_index(['Date','State'])
    X_temp_data.index = X_temp_data.index.set_names('Target_day', level=0)
    
    conf_timeseries_data = create_timeseries(data,"Confirmed_diff")
    X_conf_data, Y_data  = convert_timeseries_to_data(conf_timeseries_data, 
                                                window_size, 
                                                col_name="Confirmed_Diff")
    
    X_data = X_conf_data.merge(X_temp_data, how="left", left_index=True, right_index=True)

    if test_data:
        return X_data
    else: 
        return (X_data, Y_data)

def model_10_processing(train, val):
    model = LinearRegression()
    X_train, Y_train = model_10_pipeline(train)
    X_val, Y_val     = model_10_pipeline(val)
    model.fit(X_train, Y_train)
    return (model, (X_train, Y_train), (X_val, Y_val))

In [872]:
with_state_avg = weather_train.merge(state_avg_temps, how="left", left_on='State',right_index=True)


In [881]:

with_state_avg['Diff from Avg'] = with_state_avg['Monthly Avg Temp (F)'] - with_state_avg['State Avg Temp']
with_state_avg['Diff below Avg'] = with_state_avg['Diff from Avg'].apply(zero_positive_values)
temp = with_state_avg['Diff below Avg']
print("Positive diffs remaining: ", temp[temp>0].shape[0])

Positive diffs remaining:  0
